In [1]:
from delta import *
import pyspark
from pyspark.sql import SparkSession

delta_spark_version = "2.1.0"
spark_jars_packages = f"io.delta:delta-core_2.12:{delta_spark_version}"

spark = (
            SparkSession.builder.master("local[*]")
                .appName("PySparkLocal")
                .config("spark.executor.memory", "4g")
                .config("spark.driver.memory", "2g")
                .config("spark.memory.offHeap.enabled", True)
                .config("spark.memory.offHeap.size","8g")
                .config("spark.jars.packages", spark_jars_packages)
                .config("spark.sql.sources.partitionOverwriteMode", "dynamic")
                .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
                .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
                .config("spark.databricks.delta.schema.autoMerge.enabled", "true")
                .config("spark.databricks.delta.retentionDurationCheck.enabled", "false")
                .config("spark.databricks.delta.optimize.repartition.enabled", "true")
                .getOrCreate()
        )

:: loading settings :: url = jar:file:/Users/pii/Library/Caches/pypoetry/virtualenvs/delta-demo-1o0m9wMe-py3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/pii/.ivy2/cache
The jars for the packages stored in: /Users/pii/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-212ac570-960f-4b67-b6b5-2219323f0dd8;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 192ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted

22/10/14 15:34:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [9]:
deltaDf = spark.read.format("delta").load('../Deltalake/crypto')
deltaDf.count()

30056370

In [10]:
deltaDf.select('isoDate').distinct().count()

15

In [11]:
deltaDf.show()

+---------+--------+--------+--------------+-------------+------------+-----------+-----------+----+----------+
|  tradeId|   price|     qty|      quoteQty|         time|isBuyerMaker|isBestMatch|tradingPair|year|   isoDate|
+---------+--------+--------+--------------+-------------+------------+-----------+-----------+----+----------+
|694889278|53611.41|0.009328|  500.08723248|1615354518394|        true|       true|    BTCUSDT|2021|2021-03-10|
|694889279|53611.41| 0.02614|  1401.4022574|1615354518462|        true|       true|    BTCUSDT|2021|2021-03-10|
|694889280|53611.42| 1.95E-4|    10.4542269|1615354518481|       false|       true|    BTCUSDT|2021|2021-03-10|
|694889281|53611.41|0.011533|  618.30039153|1615354518689|        true|       true|    BTCUSDT|2021|2021-03-10|
|694889282|53611.42|  9.1E-4|    48.7863922|1615354518795|       false|       true|    BTCUSDT|2021|2021-03-10|
|694889283|53611.42|0.007399|  396.67089658|1615354519113|       false|       true|    BTCUSDT|2021|2021

In [12]:
from pyspark.sql.functions import avg

deltaDf.select(avg(deltaDf.price)).show()

+----------------+
|      avg(price)|
+----------------+
|52848.8452207497|
+----------------+



In [64]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, '../Deltalake/crypto')
deltaTable.history().show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|     32|2022-10-11 14:58:...|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|         31|  Serializable|        false|{numFiles -> 12, ...|        null|Apache-Spark/3.3....|
|     31|2022-10-11 14:58:...|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|         30|  Serializable|        false|{numFiles -> 12, ...|        null|Apache-Spark/3.3....|
|     30|2

In [71]:
asOfDf = spark.read.format("delta").option('timestampAsOf', '2022-10-11 14:56:00').load('../Deltalake/crypto')

28226601